In [ ]:
# -*- coding: utf-8 -*-
"""
Monitor de PEA (Población Económicamente Activa) - INEGI
Versión Final: Diseño Limpio, Adaptativo y con Tabla de Datos
"""

import time
import logging
import requests
import pandas as pd
import plotly.graph_objects as go
import streamlit as st

# === CONFIGURACIÓN ===
# Tu token personal validado
TOKEN = "ab6744a6-fdeb-064f-1034-691048c15ab0"

# === ESTRATEGIA DE CANDIDATOS ===
# Lista de IDs a probar si el principal falla (Original, Desestacionalizada, Nueva ENOE)
CANDIDATE_IDS = ["289244", "444005", "601614"]

# Rutas de API a probar por cada ID (Prioridad a la federada BIE-BISE)
API_ROUTES = [
    "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{ID}/es/00/false/BIE-BISE/2.0/{TOKEN}?type=json",
    "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{ID}/es/00/false/BIE/2.0/{TOKEN}?type=json",
    "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{ID}/es/00/false/BISE/2.0/{TOKEN}?type=json"
]

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger("INEGI-PEA")

# ------------------------ LÓGICA DE EXTRACCIÓN (ROBUSTA) ------------------------

@st.cache_data(ttl=3600, show_spinner=False)
def get_pea_data():
    """
    Intenta obtener datos iterando sobre IDs y Rutas hasta tener éxito.
    """
    session = requests.Session()
    session.headers.update({"User-Agent": "Mozilla/5.0 (INEGI-PEA-Monitor/1.0)"})
    report = []

    for indicador in CANDIDATE_IDS:
        for route_template in API_ROUTES:
            url = route_template.format(ID=indicador, TOKEN=TOKEN)
            try:
                r = session.get(url, timeout=15)
                if r.status_code == 200:
                    data = r.json()
                    # Validar que existan series y observaciones reales
                    if "Series" in data and len(data["Series"]) > 0:
                        obs = data["Series"][0].get("OBSERVATIONS")
                        if obs and len(obs) > 5:
                            return data, indicador
                        else:
                            report.append(f"ID {indicador}: 200 OK pero sin observaciones.")
                    else:
                        report.append(f"ID {indicador}: 200 OK pero JSON vacío.")
                else:
                    report.append(f"ID {indicador}: Error HTTP {r.status_code}")
            except Exception as e:
                report.append(f"ID {indicador}: Excepción {str(e)}")

            # Pequeña pausa para no saturar
            time.sleep(0.2)

    # Si termina el bucle sin retorno, lanzamos error con reporte
    raise RuntimeError(f"No se pudo obtener ninguna serie.\nDetalle:\n" + "\n".join(report))

def process_data(json_data):
    """Limpia y estructura los datos JSON en un DataFrame"""
    series_obj = json_data["Series"][0]
    observations = series_obj.get("OBSERVATIONS", [])
    rows = []

    for o in observations:
        periodo_str = o.get("TIME_PERIOD", "")
        valor_str = o.get("OBS_VALUE", "")

        if not periodo_str or not valor_str:
            continue

        try:
            # Esperamos formato Año/Periodo (ej: 2024/01)
            parts = periodo_str.split("/")
            year = int(parts[0])
            sub = int(parts[1])

            # Filtrar solo datos trimestrales estándar (1-4)
            if sub <= 4:
                rows.append({
                    "fecha": f"{year}-T{sub}",
                    "anio": year,
                    "trim": sub,
                    "valor": float(valor_str.replace(",", ""))
                })
        except:
            pass

    if not rows:
        raise ValueError("Datos vacíos después del procesamiento.")

    df = pd.DataFrame(rows)
    df.sort_values(["anio", "trim"], inplace=True)
    return df

# ------------------------ VISUALIZACIÓN AJUSTADA ------------------------

def plot_pea_clean(df):
    ultimo = df.iloc[-1]
    penultimo = df.iloc[-2] if len(df) > 1 else ultimo

    # Calcular variaciones
    var_trim = ((ultimo['valor'] / penultimo['valor']) - 1) * 100

    fig = go.Figure()

    # Barra principal
    fig.add_trace(go.Bar(
        x=df['fecha'],
        y=df['valor'],
        name="Personas",
        hovertemplate="%{x}<br><b>%{y:,.0f}</b> personas<extra></extra>"
    ))

    # Layout limpio con los ajustes solicitados
    fig.update_layout(
        title=dict(
            text="<b>Población Económicamente Activa (PEA)</b>",
            x=0.5,           # Título Centrado
            xanchor="center",
            font=dict(size=20)
        ),
        plot_bgcolor="rgba(0,0,0,0)",  # Fondo transparente
        paper_bgcolor="rgba(0,0,0,0)", # Fondo transparente
        height=550,
        margin=dict(t=80, l=50, r=50, b=100), # Margen inferior amplio para etiquetas verticales y fuente
        yaxis=dict(
            showgrid=True,
            gridcolor="#e6e6e6",
            tickformat=",",
            title="Personas"
        ),
        xaxis=dict(
            tickangle=-90,   # Etiquetas Verticales
            title=""
        )
    )

    # Anotación del último valor (SIN RECUADRO)
    # Color dinámico para la variación (Verde si sube, Rojo si baja)
    color_var = "#00c853" if var_trim > 0 else "#d50000"

    fig.add_annotation(
        x=ultimo['fecha'],
        y=ultimo['valor'],
        text=f"<b>{ultimo['valor']:,.0f}</b><br><span style='color:{color_var}; font-size:12px'>{var_trim:+.1f}% vs prev.</span>",
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=1,
        ax=0,
        ay=-60, # Subimos un poco más la etiqueta
        bgcolor="rgba(0,0,0,0)",       # Fondo transparente
        bordercolor="rgba(0,0,0,0)"    # Borde transparente
    )

    # Fuente en la parte inferior izquierda
    fig.add_annotation(
        xref="paper",
        yref="paper",
        x=0,
        y=-0.25, # Posición ajustada para evitar choque con eje X
        text="Fuente: INEGI",
        showarrow=False,
        font=dict(size=11, color="gray"),
        xanchor="left"
    )

    return fig, ultimo['valor'], var_trim

# ------------------------ APP STREAMLIT ------------------------

def main():
    st.set_page_config(page_title="Monitor PEA", page_icon="📈", layout="wide")

    st.title("🦅 Monitor del Mercado Laboral")
    st.markdown("Tablero en tiempo real conectado a la API del INEGI.")

    # Contenedor principal
    with st.container():
        try:
            with st.spinner("Conectando con servidores del INEGI..."):
                json_data, used_id = get_pea_data()
                df = process_data(json_data)

                fig, last_val, var_pct = plot_pea_clean(df)

                # KPIs Superiores (Solo 2 columnas, eliminada la de ID)
                k1, k2 = st.columns(2)
                k1.metric("PEA Total (Personas)", f"{last_val:,.0f}")
                k2.metric("Variación Trimestral", f"{var_pct:+.2f}%", delta_color="normal")

                # Gráfica
                st.plotly_chart(fig, use_container_width=True)

                # --- SECCIÓN DE TABLA DE DATOS ---
                st.markdown("### Datos Detallados")
                
                # Preparamos los datos para la tabla:
                # 1. Hacemos una copia para no alterar el original
                # 2. Ordenamos descendente (más reciente arriba) para facilitar lectura tabular
                df_table = df.copy().sort_values(["anio", "trim"], ascending=False)
                
                # Seleccionamos y renombramos columnas para presentación
                df_table = df_table[["fecha", "valor"]]
                
                st.dataframe(
                    df_table,
                    use_container_width=True,
                    hide_index=True,
                    column_config={
                        "fecha": st.column_config.TextColumn("Periodo"),
                        "valor": st.column_config.NumberColumn(
                            "Población Económicamente Activa",
                            format="%,d" # Formato entero con separador de miles
                        )
                    }
                )

        except Exception as e:
            st.error("⚠️ No se pudieron cargar los datos")
            st.code(str(e))
            st.info("Intenta recargar la página. La API del INEGI puede tener intermitencias temporales o el token haber excedido su cuota.")

if __name__ == "__main__":
    main()